In [1]:
from dataclasses import dataclass
from enum import Enum, auto
from typing import Self
from uuid import uuid4
from bamt.preprocessors import Preprocessor
import pandas as pd
from sklearn import preprocessing as pp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import re
from bamt.networks import HybridBN
import random
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from tqdm import tqdm

YEAR = 2022

In [2]:
df = pd.read_csv(

    "/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/migforecasting/mig whereabouts/inflow LO.csv")

Выделим ОКТМО из Ленобласти

In [3]:
oktmos = map(str,  df.oktmo.unique())

In [4]:
oktmos_lo = [x for x in oktmos if re.fullmatch(r"41\d+", x)]

In [5]:
del oktmos

In [6]:
len(oktmos_lo)

226

Определим численность населения. Проблема: удалось найти данные только за 2024 год

In [7]:
pop_size_df = pd.read_excel(

    "/Users/pishchulov/edu/matobes/НИР/outer_source/численность МО/Сhisl_MO_01-01-2024_only_LO.xlsx")

In [8]:
pop_size_df

,Коды территорий,Unnamed: 1,население,городское,сельское
0,4100000000,Ленинградская область,2035762,1373533,662229
1,41754000 0 0,Сосновоборский городской округ,63462,63462,0
2,41754000001 1 0 0 0,г Сосновый Бор,63462,63462,0
3,41603000 0 0,Бокситогорский муниципальный район,50855,39107,11748
4,41603101 0 0,Городское поселение Бокситогорское,15764,15480,284
...,...,...,...,...,...
271,41648418 0 0,Сельское поселение Нурминское,3338,0,3338
272,41648430 0 0,Сельское поселение Лисинское,1875,0,1875
273,41648443 0 0,Сельское поселение Тельмановское,26781,0,26781
274,41648444 0 0,Сельское поселение Трубникоборское,1599,0,1599


In [9]:
def discard_to_8_chars(s):
    if len(s) != 10:
        return s
    if s[-2:] == "00":
        return s[:-2]

In [10]:
pop_size_df["Коды территорий"] = pd.Series([discard_to_8_chars(x.replace(

    " ", "")) for x in map(str, pop_size_df["Коды территорий"])], dtype=str)

In [11]:
# pop_size_df_lo = pop_size_df[pop_size_df["Коды территорий"].isin(oktmos_lo)]

In [12]:
# set(oktmos_lo) - set(pop_size_df_lo["Коды территорий"])

Проблема: некоторые ОКТМО исчезли, потому что они эти населенные пункту переехали на новые коды. Проигнорируем.

In [13]:
del pop_size_df

In [14]:
# pop_size_df_lo=pop_size_df_lo.drop(["городское", "сельское"], axis = 1)
# pop_size_df_lo=pop_size_df_lo.set_axis(["oktmo", "name", "pop_size"], axis=1)
# pop_size_df_lo.sort_values("oktmo", inplace=True)
# pop_size_df_lo

Удалим муниципальные районы, потому что это агрегаты других МО

In [15]:
def is_mun_district(oktmo):
    assert len(oktmo) == 8
    return oktmo[-3:] == "000"

In [16]:
# pop_size_df_lo.pop_size.sum()

In [17]:
# mun_districts_lo = pop_size_df_lo[pop_size_df_lo.oktmo.apply(is_mun_district)]
# mun_districts_lo

In [18]:
# pop_size_df_lo=pop_size_df_lo[~pop_size_df_lo.oktmo.apply(is_mun_district)]
# pop_size_df_lo.pop_size.sum()

### Подготовим данные о среде. 

In [19]:
allmun = pd.read_csv(

    "/Users/pishchulov/edu/matobes/НИР/outer_source/rosstat_allmun/allmuns/popsize (allmun).csv")
allmun[["oktmo"]] = allmun[["oktmo"]].astype(str)
allmun["from_lo"] = [bool(re.fullmatch(r"41\d+", x)) for x in allmun.oktmo]
allmun = allmun[(allmun.from_lo) & (allmun.year == 2022)]
allmun = allmun[["oktmo", "name", "year", "popsize"]].set_index("oktmo")
allmun["popsize"] = allmun["popsize"].astype(int)

In [20]:
allmun

,name,year,popsize
oktmo,,,
41603000,Бокситогорский муниципальный район,2022,47236
41603101,Бокситогорское,2022,14606
41603102,Пикалёвское,2022,19250
41603155,Ефимовское,2022,4389
41603412,Большедворское,2022,1486
...,...,...,...
41648430,Лисинское,2022,1703
41648443,Тельмановское,2022,13191
41648444,Трубникоборское,2022,1479


Нет ITMO2 данных за 2023 год, поэтому используем 2022.

In [21]:

mo_features = allmun
# files = [
#     "shoparea",
#     "foodseats",
#     "agrprod",
#     "beforeschool",
#     "schoolnum",  # тут нет требуемых районов
#     "museums",
#     "theatres",
#     "musartschool",
#     "hospitalcap",
#     "pollutionvol",
#     "popsize",
#     "retailturnover",
#     "livarea",

# ]
files = """parks 
schoolnum 
cliniccap 
sportsvenue 
consnewareas 
popsize 
livestock 
saldo internat 
consnewapt 
budincome 
musartschool 
foodservturnover 
goodcompanies 
invest 
factoriescap 
shoparea 
museums 
agrprod 
roadslen 
hospitals 
outflow 
foodseats 
servicesnum 
avgsalary 
harvest 
funds 
library 
cultureorg 
saldo interreg 
avgemployers 
naturesecure 
goodcompincome 
visiblecompanies 
retailturnover 
beforeschool 
theatres 
saldo reg 
docsnum 
livarea 
badcompanies """.split('\n')

files_about_mo = ["sportsvenue", "invest",

                  "popsize", "shoparea", "roadslen", "hospitals"]
# for f in files_about
#         _mo:
#     d = pd.read_csv(
#         f"/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/migforecasting/superdataset/features separately/{f} (allmun).csv")
#     d[["oktmo"]] = d[["oktmo"]].astype(str)
#     d["from_lo"] = [bool(re.fullmatch(r"41\d+", x)) for x in d.oktmo]
#     # d_lo = d[(d.from_lo) & (d.year == 202 2 (
#         ]
#     # print(f == f} {len(d_lo)}")
#     d = d[(d.oktmo.isin(mo_features.index)) & (
#         d.year == 2022)].drop(["from_lo"], axis=1)
#     if len(d) == 0:
#         print(f"error in {f}")
#     else:
#         mo_features = mo_features.join(d.set_index("oktmo").iloc[:, -1:], )

In [22]:
mun_districts = mo_features[pd.Series(
    mo_features.index, index=mo_features.index).apply(is_mun_district)]["name"]
mun_districts

oktmo
41603000     Бокситогорский муниципальный район
41606000        Волосовский муниципальный район
41609000         Волховский муниципальный район
41612000       Всеволожский муниципальный район
41615000         Выборгский муниципальный район
41618000         Гатчинский муниципальный район
41621000      Кингисеппский муниципальный район
41624000          Киришский муниципальный район
41625000          Кировский муниципальный район
41627000    Лодейнопольский муниципальный район
41630000      Ломоносовский муниципальный район
41633000            Лужский муниципальный район
41636000       Подпорожский муниципальный район
41639000        Приозерский муниципальный район
41642000        Сланцевский муниципальный район
41645000         Тихвинский муниципальный район
41648000         Тосненский муниципальный район
41754000                         Сосновоборский
Name: name, dtype: object

#### Список муниципальных образований без муниципальных округов

In [23]:
mun_units = mo_features[~pd.Series(
    mo_features.index, index=mo_features.index).apply(is_mun_district)][["name", "popsize"]]

In [24]:
len(mo_features), len(mun_units)

(205, 187)

# Генерация популяции

Распределения по полу и возрасту есть только для муниципальных районов.
migration/ITMO-2/popsize/data0.xlsx

In [25]:
@dataclass
class AgeGroup:
    lo: int
    hi: int

    def __str__(self):
        return f"{self.lo}-{self.hi}"

    @staticmethod
    def from_str(s):
        lo, hi = list(map(int, s.split('-')))
        return AgeGroup(lo=lo, hi=hi)

    def get_random_age(self):
        return random.choice(list(range(self.lo, self.hi+1)))


age_groups = [
    AgeGroup(0, 0),
    AgeGroup(1, 4),
    AgeGroup(5, 9),
    AgeGroup(10, 14),
    AgeGroup(15, 19),
    AgeGroup(20, 24),
    AgeGroup(25, 29),
    AgeGroup(30, 34),
    AgeGroup(35, 39),
    AgeGroup(40, 44),
    AgeGroup(45, 49),
    AgeGroup(50, 54),
    AgeGroup(55, 59),
    AgeGroup(60, 64),
    AgeGroup(65, 69),
    AgeGroup(70, 74),
    AgeGroup(75, 79),
    AgeGroup(80, 84),
    AgeGroup(85, 89),
    AgeGroup(90, 94),
    AgeGroup(95, 99),
    AgeGroup(100, 100),
]
gender_age_dist_by_district_oktmo = {}
for mun_district_oktmo, mun_district_name_series in pd.DataFrame(mun_districts).iterrows():
    mun_district_name = mun_district_name_series.iloc[0]
    mun_district_first_name = mun_district_name.split()[0]
    try:
        df = pd.read_excel(

            "/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/popsize/data0.xlsx", sheet_name=mun_district_first_name)
    except ValueError:
        df = pd.read_excel(

            "/Users/pishchulov/edu/matobes/НИР/outer_source/migration/ITMO-2/popsize/data0.xlsx", sheet_name=mun_district_first_name+" ")

    col_num = list(df.iloc[0, :]).index(YEAR)
    year_data = df.iloc[:, col_num]
    males = []
    females = []
    for i, ag in enumerate(age_groups):
        female = year_data.iloc[6+i*4]
        male = year_data.iloc[7+i*4]
        males.append(male)
        females.append(female)

    gender_age_dist = pd.DataFrame(
        {"male": males, "female": females}, index=list(map(str, age_groups)))
    gender_age_dist = gender_age_dist.fillna(gender_age_dist.mean())
    gender_age_dist_by_district_oktmo[mun_district_oktmo] = gender_age_dist

##### Построим набор данных, следующих имеющемуся распредление пола и возраста

In [26]:
def gender_age_dist_to_population(gender_age_dist_by_district_oktmo):
    """Принимает распределение по полу и возрасту для каждого мун. района.
    Генереириует популяцию.
    Разворачивает статистики в выборку, 
    предполагая, что в каждой возрастной группе возраст распредлен равномерно"""
    age_sex_population = []
    for district_oktmo, age_sed_distribution_df in gender_age_dist_by_district_oktmo.items():
        for age_group_str, row in age_sed_distribution_df.iterrows():
            age_group = AgeGroup.from_str(age_group_str)
            males_cnt = int(row["male"])
            females_cnt = int(row["female"])
            for _ in range(males_cnt):
                age_sex_population.append(
                    (age_group.get_random_age(), 1, district_oktmo))
            for _ in range(females_cnt):
                age_sex_population.append(
                    (age_group.get_random_age(), 0, district_oktmo))
    population = pd.DataFrame(age_sex_population, columns=[
        "age", "male", "district_oktmo"]).sample(frac=1).reset_index(drop=True)
    return population

Популяция по мун. районам

In [27]:
population = gender_age_dist_to_population(gender_age_dist_by_district_oktmo)
mun_district_population = population
population

,age,male,district_oktmo
0,20,1,41612000
1,33,1,41621000
2,13,0,41612000
3,20,0,41612000
4,66,0,41624000
...,...,...,...
2074072,32,1,41612000
2074073,35,0,41618000
2074074,51,0,41612000
2074075,1,0,41618000


### Определим образование. Построим условное распределение образования от пола и возраста.

In [51]:
rlms = pd.read_excel("/Users/pishchulov/edu/matobes/НИР/Data_RLMS.xlsx")
data = rlms[["age", "male", "educ", "lnwage", "children"]]

#### Обучим классифкатор для предсказания уровня образования на данных RLMS

In [52]:
X_train, X_test, y_train, y_test = train_test_split(data[["age", "male"]], data['educ'], test_size=.2)
educ_clsf = XGBClassifier()
educ_clsf.fit(X_train, y_train)
preds = educ_clsf.predict(X_test) # TODO predict proba
# accuracy == 40% - на первый взгляд довольно плохой результат, 
# но все равно гораздо лучше, чем назначать уровень образования исходя из общих рассуждений и здравого смысла
accuracy_score(y_test, preds)

0.4206451612903226

#### Применим классификатор, чтобы назначить уровень образования для агентов из популяции

In [53]:
population["educ"] =  educ_clsf.predict(population[["age", "male"]])

In [54]:
population

,age,male,district_oktmo,educ,lnwage,children
0,20,1,41612000,2,9.771714,0
1,33,1,41621000,3,10.260056,1
2,13,0,41612000,1,9.221813,0
3,20,0,41612000,2,9.401896,0
4,66,0,41624000,2,8.994318,3
...,...,...,...,...,...,...
2074072,32,1,41612000,3,10.072206,1
2074073,35,0,41618000,3,9.877120,2
2074074,51,0,41612000,2,9.862335,1
2074075,1,0,41618000,1,9.221813,0


### Определим зарплату. Построим зависимость от пола, возраста и образования

In [32]:
X_train, X_test, y_train, y_test = train_test_split(data[["age", "male", "educ"]], data['lnwage'], test_size=.2)
wage_regressor = XGBRegressor()
wage_regressor.fit(X_train, y_train)
preds = wage_regressor.predict(X_test)
mape(y_test, preds) # mape == 0.05 - отличный результат. 

0.05854051029013592

In [33]:
population["lnwage"] = wage_regressor.predict(population[["age", "male", "educ"]])

### Определим количество детей

In [34]:
X_train, X_test, y_train, y_test = train_test_split(data[["age", "male", "educ", "lnwage"]], data["children"], 
                                                    test_size=.2)
children_regressor = XGBRegressor()
children_regressor.fit(X_train, y_train)
preds = children_regressor.predict(X_test).round()
mae(y_test, preds) 

0.727741935483871

In [35]:
population["children"] = children_regressor.predict(population[["age", "male", "educ", "lnwage"]]).round().astype(int)

In [36]:

data["male"] = rlms.male.astype(str)
data["educ"] = rlms.educ.astype(str)

# set encoder and discretizer
encoder = pp.LabelEncoder()
discretizer = pp.KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="uniform")

# create preprocessor object with encoder and discretizer
p = Preprocessor([("encoder", encoder), ("discretizer", discretizer)])

# discretize data for structure learning
discretized_data, est = p.apply(data)

# get information about data
info = p.info

# initialize network object
bn = HybridBN(use_mixture=True, has_logit=True)

# add nodes to network
bn.add_nodes(info)

# using mutual information as scoring function for structure learning
bn.add_edges(
    discretized_data,
    #  scoring_function=('MI',)
)

# or use evolutionary algorithm to learn structure

bn.add_edges(discretized_data)

bn.fit_parameters(data)

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [37]:
# bn.save("pop.json")
bn.plot('mixture.html')
bn.validate(info)


True

In [38]:
sampled_data = bn.sample(10_000, progress_bar=False)
# sampled_data.children = sampled_data.children.astype(int)
sampled_data.male = sampled_data.male.astype(int)
sampled_data.age = sampled_data.age.astype(int)

In [39]:
sampled_data

,lnwage,children,male,educ,age
0,10.434318,1,1,2,32
1,10.446630,0,0,3,35
2,9.653095,1,0,2,46
3,10.758057,2,0,1,49
4,10.426513,1,1,1,43
...,...,...,...,...,...
9995,11.071176,2,1,1,30
9996,9.856036,1,0,3,30
9997,9.241148,2,0,1,47
9998,9.634657,2,0,3,34


# Сгенерирруем популяцию для каждого МО из Ленобласти

In [40]:
def to_district_oktmo(oktmo):
    assert len(oktmo) == 8
    return oktmo[:5]+"000"

In [57]:
from functools import cache

def get_mun_unit_population(mun_units, mun_district_population):
    mun_unit_population = pd.DataFrame(columns=["age", "male", "oktmo"])
    for mun_unit in tqdm(list(mun_units.reset_index().itertuples(index=False))):
        district_oktmo = to_district_oktmo(mun_unit.oktmo)
        pop = (mun_district_population[mun_district_population.district_oktmo == district_oktmo]
            .sample(mun_unit.popsize)[["age", "male"]])
        pop["oktmo"] = mun_unit.oktmo
        pop = pd.DataFrame(pop, columns=["age", "male", "oktmo"])
        mun_unit_population = pd.concat([mun_unit_population, pop])
    mun_unit_population["age"] = mun_unit_population["age"].astype(int)
    mun_unit_population["male"] = mun_unit_population["male"].astype(int)
    mun_unit_population["oktmo"] = mun_unit_population["oktmo"].astype(str)
    return mun_unit_population

mun_unit_population = get_mun_unit_population(mun_units, mun_district_population)

100%|██████████| 187/187 [00:20<00:00,  9.22it/s]


In [58]:
mun_unit_population

,age,male,oktmo
148987,71,0,41603101
128771,76,1,41603101
1381512,62,0,41603101
1164120,75,0,41603101
1121637,51,1,41603101
...,...,...,...
410960,17,1,41648464
1763266,76,0,41648464
1310280,34,1,41648464
1016472,51,0,41648464


### Определим образование, зарплату, количество детей для популяции

In [59]:
mun_unit_population["educ"] =  educ_clsf.predict(mun_unit_population[["age", "male"]])
mun_unit_population["lnwage"] = wage_regressor.predict(mun_unit_population[["age", "male", "educ"]])
mun_unit_population["children"] = children_regressor.predict(mun_unit_population[["age", "male", "educ", "lnwage"]]).round().astype(int)
